In [ ]:
!git clone https://github.com/lushan88a/google_trans_new

Cloning into 'google_trans_new'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 216 (delta 49), reused 45 (delta 45), pack-reused 165
Receiving objects: 100% (216/216), 97.37 KiB | 1.39 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
from google_trans_new.google_trans_new import google_translator
import pandas as pd
import numpy as np

In [ ]:
translator = google_translator()
translate_text = translator.translate('منتج جيد',lang_tgt='en')  
print(translate_text)

A good product 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
df = pd.read_csv('spam.csv', delimiter = ',',encoding='latin-1')

In [ ]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df = df.rename(columns = {'v1':'spam',
                     'v2':'text'})

In [ ]:
df.head()

,spam,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], axis = 1, inplace = True)

In [ ]:
df

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
x = df.text
y = df.spam
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
  inputs = Input(name='inputs', shape = [max_len])
  layer = Embedding(max_words, 50, input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256, name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1, name ='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs, outputs = layer)
  return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer = RMSprop(), metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [ ]:
model.fit(sequences_matrix, y_train,
          batch_size = 128,
          epochs= 10,
          validation_split = 0.2)

Epoch 1/10
30/30 [==============================] - 14s 358ms/step - loss: 0.3263 - accuracy: 0.8778 - val_loss: 0.1380 - val_accuracy: 0.9557
Epoch 2/10
30/30 [==============================] - 10s 333ms/step - loss: 0.0844 - accuracy: 0.9823 - val_loss: 0.0662 - val_accuracy: 0.9821
Epoch 3/10
30/30 [==============================] - 6s 191ms/step - loss: 0.0402 - accuracy: 0.9894 - val_loss: 0.0566 - val_accuracy: 0.9842
Epoch 4/10
30/30 [==============================] - 6s 192ms/step - loss: 0.0331 - accuracy: 0.9913 - val_loss: 0.0399 - val_accuracy: 0.9863
Epoch 5/10
30/30 [==============================] - 6s 193ms/step - loss: 0.0239 - accuracy: 0.9923 - val_loss: 0.0552 - val_accuracy: 0.9800
Epoch 6/10
30/30 [==============================] - 6s 190ms/step - loss: 0.0190 - accuracy: 0.9942 - val_loss: 0.0579 - val_accuracy: 0.9821
Epoch 7/10
30/30 [==============================] - 6s 191ms/step - loss: 0.0142 - accuracy: 0.9955 - val_loss: 0.0635 - val_accuracy: 0.9821
Epoc

In [ ]:
test_seq = tok.texts_to_sequences(X_test)
test_seq_matrix = pad_sequences(test_seq, maxlen=max_len)

In [ ]:
accuracy = model.evaluate(test_seq_matrix, y_test)

27/27 [==============================] - 1s 23ms/step - loss: 0.1001 - accuracy: 0.9809


In [ ]:
print("Test's Loss : ", accuracy[0], " Acc: ", accuracy[1])

Test's Loss :  0.10010074824094772  Acc:  0.980861246585846
